In [37]:
#Run this first

from pymatgen import electronic_structure #Make sure that pymatgen is installed
from pymatgen.electronic_structure import plotter
from pymatgen.electronic_structure.plotter import BSDOSPlotter, BSPlotter, DosPlotter
from mp_api.client import MPRester

My_API = 'MJUC3rwGsmc3tEkuE5Q5gOkT3prWnpad' #Find this API Key by going to materialsproject.org -> API -> API Key. (You need an account to do this)

Mat_ID = "mp-763" #Every material in materialsproject.org has a unique ID attribute to it. Enter this ID here

with MPRester(api_key=My_API) as mpr:
    docs = mpr.summary.search(elements=["Cu", "O"]) #docs contains information about all materials in materials project that contain Cu and O.
#print(docs[0].material_id) 
print('Number of matching compounds from materials project: '+str(len(docs)))

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/mpcontribs/client/__init__.py:90: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.set_option('mode.use_inf_as_na', True)
Accessing summary data through MPRester.summary is deprecated. Please use MPRester.materials.summary instead.
Retrieving SummaryDoc documents: 100%|██████████| 4957/4957 [00:11<00:00, 441.51it/s]

mp-2219075
4957


In [73]:
import re
Caps = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

def expand(f): #Expand a chemical formula to get rid of the parentheses
    nums = ['0','1','2','3','4','5','6','7','8','9']

    res = re.search('\((.*)\)',f) #Find the group within the parentheses
    try:
        ss = res.group(1) #Are there any parentheses?
    except:
        return(f) # ... If not, just return the input
    
    res2 = re.search('(.*)\(',f) #Redefine the new string (subs2) such that it begins with the characters preceding the parentheses.
    subs2 = res2.group(1) #subs2 is what the function returns

    #Find the multiplier that follows the parentheses
    ####
    i = int(1)
    l = len(f)
    num = int(0)
    while(1):
        if f[l-i] not in nums:
            break
        else:
            num += 10**(i-1)*int(f[l-i])
            i += 1
    mult = str(num) #Multiplier in string format
    ####

    #Break the terms up that are inside the parentheses
    ####
    s = 0
    subs = [] #Each item in subs is an element along with its stoichiometric factor
    for t in range(len(ss)):
        if t == 0:
            continue
        if ss[t] in Caps:
            subs.append(ss[s:t])
            s = t
        if t == (len(ss)-1):
            subs.append(ss[s:t+1])
    ####

    #Modify each term according to the multiplier and add it back to the base string (what comes before the parentheses)
    ####
    for sub in subs:
        i = int(1)
        l = len(sub)
        num = int(0) #New number for each term
        if sub[l-1] not in nums:
            sub += mult

        else:
            while(1):
                if sub[l-i] not in nums:
                    i -= 1
                    break
                else:
                    num += 10**(i-1)*int(sub[l-i])
                    i += 1 
            sub = sub[:-i]+str(num*int(mult)) #Rewrite each elemental term
        subs2 += sub #Assemble the expanded chemical formula
    ####

    return(subs2)

mats = [] #mats stores the expanded chemical formulae of all relevant materials
for doc in docs:
    formula = doc.formula_pretty
    ID = doc.material_id
    
    #The optional commented code below is an optional filter that only considers materials that are theoretical metals (this includes mott insulators)
    """
    if doc.is_metal: #Only consider compounds which are theoretically metallic
        mats.append(expand(formula))
    """
    mats.append(expand(formula)) #Comment out this line if the above section is uncommented.

    #The optional commented code below prints some example chemical formulae before and after filtering (if they don't match, then they didn't get through the filter)
    """
    if ID == 'mp-555855' or ID == 'mp-23102': #Two examples 
        print(formula)
        print(mats[-1])
    """

Sr2Cu(ClO)2
Sr2CuCl2O2
Sr2CaCu2(BiO4)2
Cu2AsO5


In [76]:
from itertools import permutations
#Annoyingly, the 2D materials database and materials project don't have the same naming conventions for certain compounds.
#This means that we have to consider all possible permutations in writing the chemical formulae, which is what the code 
#below accomplishes.

mats2 = [] #This is the new list where all permutations are stored.

for mat in mats: #Iterate over all compounds
    s = 0
    subs = []
    for t in range(len(mat)): #Find each element and its stoichiometric factor. This is stored in 'subs'
        if t == 0:
            continue
        if mat[t] in Caps: 
            subs.append(mat[s:t])
            s = t
        elif t == (len(mat)-1):
            subs.append(mat[s:t+1])
    #print(subs)
    Sl = len(subs)
    P = list(permutations(range(Sl))) #List all possible permutations for the chemical formula
    for p in P:
        out = ''
        for y in range(Sl):
            out += subs[p[y]]
        mats2.append(out)
print('Number of compound permutations: '+str(len(mats2)))
#print(mats2[0])
#print(mats2[1])
#print(mats2[2])

Number of compound permutations: 18174
Cu6PbO8
Cu6O8Pb
PbCu6O8


In [75]:
from M2D import * #Make sure that M2D.py is saved in the same directory as this jupyter notebook file.
M = Mat2d()
Mats = list(M.Mat['data']['compounds']) #Mats is the complete list of materials from the 2D material cloud database.
Final = [] 
for mat in mats2:
    if (mat in Mats) and (mat not in Final): #Check for any compounds that appear in both materials project and the 2D database.
        Final.append(mat) #...these compounds are stored in 'Final'
        print(mat)

Sr2CuO2Cl2
K2Cu3Se4O12
Ba2Cu2Te4O11Br2
Sr2CuO2Br2
Sr2CuO2I2
Cu3Yb2Te4O12Cl4
K2CuSO4Cl2
CuSb6S2O16
Cu2P2O7
K2CuP2O7
CuSe2O5
CuO6I2
Ca2CuO2Br2
